# Data  Exploration and Cleaning

The code starts with the cell which imports the required libraries and also used Python function. This function utilizes a library called Pandas to fetch information from three CSV files stored on GitHub and organize it into Pandas DataFrames. Additionally, we apply a setting called `low_memory=False` to the function to prevent any alerts or notifications regarding the presence of mixed data types within the columns. This setting is especially useful when handling extensive datasets.

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.exceptions import UndefinedMetricWarning 
import warnings
import seaborn as sns
import matplotlib.pyplot as plt 
from sklearn.linear_model import LogisticRegression  
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC 

#data on github url
url_accident = "https://raw.githubusercontent.com/Sayalinale/RoadTraffic-Analysis/main/dft-road-casualty-statistics-accident-2021.csv"
url_casualty = "https://raw.githubusercontent.com/Sayalinale/RoadTraffic-Analysis/main/dft-road-casualty-statistics-casualty-2021.csv"
url_vehicle = "https://raw.githubusercontent.com/Sayalinale/RoadTraffic-Analysis/main/dft-road-casualty-statistics-vehicle-2021.csv"

# Load CSV files into Pandas  DataFrames with low_memory=False
df_accident_data = pd.read_csv(url_accident, low_memory=False)
df_casualty_data = pd.read_csv(url_casualty, low_memory=False)
df_vehicle_data = pd.read_csv(url_vehicle, low_memory=False)

To know more about the size of the dataset checked the shape three csv files 

In [2]:
df_accident_data.shape

(101087, 36)

In [3]:
df_casualty_data.shape

(128209, 19)

In [4]:
df_vehicle_data.shape

(186443, 29)

As we got the output from the above cell we got the how many records and attributes are there in three different files now merging of three files would be helpful to work with the dataset. We are going to merge three files using the common column in three files which is 'accident_reference'. 

In [5]:
# Merge the DataFrames on the 'accident_reference' column
merged_data = pd.merge(df_accident_data, df_casualty_data, on='accident_reference')
merged_data = pd.merge(merged_data, df_vehicle_data, on='accident_reference')

# Now, 'merged_data' contains the merged data from all three DataFrames based on 'accident_reference'


To know about the merged files. I have printed out the shape and column name.

In [6]:
print(merged_data.columns)
print(merged_data.shape)

Index(['accident_index_x', 'accident_year_x', 'accident_reference',
       'location_easting_osgr', 'location_northing_osgr', 'longitude',
       'latitude', 'police_force', 'accident_severity', 'number_of_vehicles',
       'number_of_casualties', 'date', 'day_of_week', 'time',
       'local_authority_district', 'local_authority_ons_district',
       'local_authority_highway', 'first_road_class', 'first_road_number',
       'road_type', 'speed_limit', 'junction_detail', 'junction_control',
       'second_road_class', 'second_road_number',
       'pedestrian_crossing_human_control',
       'pedestrian_crossing_physical_facilities', 'light_conditions',
       'weather_conditions', 'road_surface_conditions',
       'special_conditions_at_site', 'carriageway_hazards',
       'urban_or_rural_area', 'did_police_officer_attend_scene_of_accident',
       'trunk_road_flag', 'trunk_road_flag.1', 'accident_index_y',
       'accident_year_y', 'vehicle_reference_x', 'casualty_reference',
       'ca

When we merged the three csv files, there are few coulmns which are similar,to confirm it we have checked values from each column. In the below given code's output it shows that accident_index and accident_year has multiple columns of same value.

In [7]:
# Check if the columns have the same values
# 1. Check columns: accident_index_x, accident_index, accident_index_y
same_accident_index = merged_data['accident_index_x'] == merged_data['accident_index_y']
same_accident_index = same_accident_index & (merged_data['accident_index_x'] == merged_data['accident_index'])

# 2. Check columns: accident_year_x, accident_year_y, accident_year
same_accident_year = merged_data['accident_year_x'] == merged_data['accident_year_y']
same_accident_year = same_accident_year & (merged_data['accident_year_x'] == merged_data['accident_year'])

# Print the results
print("1. Columns accident_index_x, accident_index, accident_index_y:")
print("All values are the same:", same_accident_index.all())

print("\n2. Columns accident_year_x, accident_year_y, accident_year:")
print("All values are the same:", same_accident_year.all())



1. Columns accident_index_x, accident_index, accident_index_y:
All values are the same: True

2. Columns accident_year_x, accident_year_y, accident_year:
All values are the same: True


In this step we will check null values for the merged file.

In [8]:
# Check for null values in the merged DataFrame
null_values = merged_data.isnull()

# Count the number of null values in each column
null_counts = null_values.sum()

# Print the null counts for all columns
for column, count in null_counts.items():
    print(f"Column '{column}': {count} missing values")


Column 'accident_index_x': 0 missing values
Column 'accident_year_x': 0 missing values
Column 'accident_reference': 0 missing values
Column 'location_easting_osgr': 43 missing values
Column 'location_northing_osgr': 43 missing values
Column 'longitude': 43 missing values
Column 'latitude': 43 missing values
Column 'police_force': 0 missing values
Column 'accident_severity': 0 missing values
Column 'number_of_vehicles': 0 missing values
Column 'number_of_casualties': 0 missing values
Column 'date': 0 missing values
Column 'day_of_week': 0 missing values
Column 'time': 0 missing values
Column 'local_authority_district': 0 missing values
Column 'local_authority_ons_district': 0 missing values
Column 'local_authority_highway': 0 missing values
Column 'first_road_class': 0 missing values
Column 'first_road_number': 0 missing values
Column 'road_type': 0 missing values
Column 'speed_limit': 0 missing values
Column 'junction_detail': 0 missing values
Column 'junction_control': 0 missing value

Now the below cell will remove the null values and the extra columns from the dataset.

In [9]:
# List of columns to keep
columns_to_keep = [
    'accident_year',
    'accident_reference',
    'location_easting_osgr',
    'location_northing_osgr',
    'longitude',
    'latitude',
    'police_force',
    'accident_severity',
    'number_of_vehicles',
    'number_of_casualties',
    'date',
    'day_of_week',
    'time',
    'local_authority_district',
    'first_road_class',
    'first_road_number',
    'road_type',
    'speed_limit',
    'junction_detail',
    'junction_control',
    'second_road_class',
    'second_road_number',
    'pedestrian_crossing_human_control',
    'pedestrian_crossing_physical_facilities',
    'light_conditions',
    'weather_conditions',
    'road_surface_conditions',
    'special_conditions_at_site',
    'carriageway_hazards',
    'urban_or_rural_area',
    'did_police_officer_attend_scene_of_accident',
    'trunk_road_flag',
    'trunk_road_flag.1',
    'vehicle_reference_x',
    'casualty_reference',
    'casualty_class',
    'sex_of_casualty',
    'age_of_casualty',
    'age_band_of_casualty',
    'casualty_severity',
    'pedestrian_location',
    'pedestrian_movement',
    'car_passenger',
    'bus_or_coach_passenger',
    'pedestrian_road_maintenance_worker',
    'casualty_type',
    'casualty_home_area_type',
    'casualty_imd_decile',
    'vehicle_type',
    'towing_and_articulation',
    'vehicle_manoeuvre',
    'vehicle_direction_from',
    'vehicle_direction_to',
    'vehicle_location_restricted_lane',
    'vehicle_location_restricted_lane.1',
    'skidding_and_overturning',
    'hit_object_in_carriageway',
    'vehicle_leaving_carriageway',
    'hit_object_off_carriageway',
    'first_point_of_impact',
    'vehicle_left_hand_drive',
    'journey_purpose_of_driver',
    'sex_of_driver',
    'age_of_driver',
    'age_band_of_driver',
    'engine_capacity_cc',
    'propulsion_code',
    'age_of_vehicle',
    'driver_imd_decile',
    'driver_home_area_type',
    'lsoa_of_driver'
]

# Keep only the specified columns
merged_data = merged_data[columns_to_keep]

# Remove rows with null values
merged_data = merged_data.dropna()



To make sure we have removed the null values and the extra columns the following two cell will print the shape and columns name.

In [10]:
merged_data.shape

(246271, 71)

In [11]:
print(merged_data.columns)

Index(['accident_year', 'accident_reference', 'location_easting_osgr',
       'location_northing_osgr', 'longitude', 'latitude', 'police_force',
       'accident_severity', 'number_of_vehicles', 'number_of_casualties',
       'date', 'day_of_week', 'time', 'local_authority_district',
       'first_road_class', 'first_road_number', 'road_type', 'speed_limit',
       'junction_detail', 'junction_control', 'second_road_class',
       'second_road_number', 'pedestrian_crossing_human_control',
       'pedestrian_crossing_physical_facilities', 'light_conditions',
       'weather_conditions', 'road_surface_conditions',
       'special_conditions_at_site', 'carriageway_hazards',
       'urban_or_rural_area', 'did_police_officer_attend_scene_of_accident',
       'trunk_road_flag', 'trunk_road_flag.1', 'vehicle_reference_x',
       'casualty_reference', 'casualty_class', 'sex_of_casualty',
       'age_of_casualty', 'age_band_of_casualty', 'casualty_severity',
       'pedestrian_location', 'pedes

In [12]:
# merged_data = merged_data[merged_data != -1].dropna()
# merged_data.shape

# Statistical information regarding traffic accident characteristics

Now data is processed and ready to do analysis. The next we will performing the descriptive statestical summary of our dataset.In which we will get count of the traffic accident according to the casuality servity (fatal,serious,slight) with the considerdation of various variables such as day of week, light consition wheather condition etc.

In [13]:
# Load CSV files into Pandas DataFrames with low_memory=False
labels_accident = "https://raw.githubusercontent.com/Sayalinale/RoadTraffic-Analysis/main/Road-Safety-Open-Dataset-Data-Guide.csv"
df_accident_data_labels = pd.read_csv(labels_accident, low_memory=False)

# Define a function to map codes to labels for a specific field
def map_codes_to_labels(field_name):
    field_labels = df_accident_data_labels[df_accident_data_labels['field name'] == field_name]
    code_to_label = dict(zip(field_labels['code/format'], field_labels['label']))
    return code_to_label

# Define a function to count occurrences for a specific field by casualty_severity
def count_field_by_severity(data, field_name):
    counts = {}
    for severity in [1, 2, 3]:
        subset = data[data['accident_severity'] == severity]
        field_counts = subset[field_name].value_counts()
        counts[severity] = field_counts
    return counts

# List of variables which should analyze
variables_to_analyze = [
    'day_of_week',
    'road_type',
    'light_conditions',
    'weather_conditions',
    'road_surface_conditions',
    'sex_of_driver',
    'casualty_class',
    'junction_detail',
    'junction_control'
]

# Iterate over each variable and create tables
for variable in variables_to_analyze:
    # Count occurrences for the current variable by casualty_severity
    variable_counts_by_severity = count_field_by_severity(merged_data, variable)

    # Get the code to label mapping for the current variable
    variable_labels = map_codes_to_labels(variable)

    # Get the label for 'accident_severity' field
    accident_severity_label = map_codes_to_labels('accident_severity')

    # Display the counts with labels
    print(f"Analysis for {variable}:")
    for severity, counts in variable_counts_by_severity.items():
        severity_label = accident_severity_label.get(str(severity), 'Unknown')
        print(f" {severity_label}:")
        for code, count in counts.items():
            label = variable_labels.get(str(code), 'Unknown')
            print(f"  {label}: {count}")
    print("\n")


Analysis for day_of_week:
 Fatal:
  Thursday: 714
  Friday: 706
  Sunday: 690
  Saturday: 684
  Wednesday: 682
  Monday: 597
  Tuesday: 594
 Serious:
  Friday: 8648
  Saturday: 8243
  Wednesday: 7426
  Monday: 7363
  Thursday: 7202
  Tuesday: 7197
  Sunday: 6975
 Slight:
  Friday: 31965
  Thursday: 28226
  Wednesday: 27683
  Tuesday: 26970
  Saturday: 26318
  Monday: 25934
  Sunday: 21454


Analysis for road_type:
 Fatal:
  Single carriageway: 3398
  Dual carriageway: 1153
  Roundabout: 51
  Slip road: 37
  Unknown: 18
  One way street: 10
 Serious:
  Single carriageway: 40043
  Dual carriageway: 9128
  Roundabout: 2079
  Slip road: 733
  One way street: 652
  Unknown: 419
 Slight:
  Single carriageway: 130070
  Dual carriageway: 34242
  Roundabout: 11561
  Unknown: 4742
  Slip road: 4140
  One way street: 3795


Analysis for light_conditions:
 Fatal:
  Daylight: 2905
  Darkness - lights lit: 831
  Darkness - no lighting: 739
  Darkness - lighting unknown: 111
  Darkness - lights unlit

In [14]:
# Calculate the correlation matrix
corr_matrix = merged_data.corr()

# Calculate the correlation between features and the target variable
corr_with_target = merged_data.corr()['accident_severity'].sort_values(ascending=False)

# Print the correlation values with the target variable one by one
for feature, correlation in corr_with_target.items():
    print(f"{feature}: {correlation}")


accident_severity: 1.0
casualty_severity: 0.8427407000572424
did_police_officer_attend_scene_of_accident: 0.1688781437143721
hit_object_in_carriageway: 0.10847302170061227
vehicle_location_restricted_lane: 0.10845346572841447
vehicle_location_restricted_lane.1: 0.10845346572841415
hit_object_off_carriageway: 0.1039373035463503
vehicle_left_hand_drive: 0.09990976872222491
junction_control: 0.09028142063434047
trunk_road_flag.1: 0.08679564487469517
trunk_road_flag: 0.08679564487469434
vehicle_manoeuvre: 0.08144323856635824
sex_of_driver: 0.08119273931047775
skidding_and_overturning: 0.07621814254790718
first_point_of_impact: 0.07604097849714683
pedestrian_crossing_physical_facilities: 0.07601843984877772
pedestrian_crossing_human_control: 0.07472257471703823
second_road_class: 0.07441180273309352
longitude: 0.0691579465471388
location_easting_osgr: 0.06812464801883984
junction_detail: 0.06567298047658213
vehicle_leaving_carriageway: 0.04529232568389992
sex_of_casualty: 0.0448719094991949

Here we can see the correlation values between the features and the target variable "accident_severity." Based on these correlation values, we can see how each feature is related to the severity of accidents. Here are some key takeaways:

Positive Correlations (values closer to 1):
- Features like "casualty_severity," "did_police_officer_attend_scene_of_accident," and "hit_object_in_carriageway" have positive correlations with accident severity. This means that as these features increase, the accident severity tends to increase as well.

Negative Correlations (values closer to -1):
- Features like "speed_limit," "number_of_casualties," "urban_or_rural_area," and others have negative correlations with accident severity. This implies that lower speed limits, fewer casualties, and accidents in urban areas are associated with less severe accidents.

Correlations close to zero indicate weaker relationships between the features and accident severity.

It's essential to interpret these correlations in the context of our analysis goals. Features with strong positive or negative correlations can be considered important factors affecting accident severity and may be valuable for building predictive models or further analysis.

These relationships are based on statistical associations, and further analysis or domain knowledge may be needed to understand the underlying causes of these correlations.

Additionally, we've noted that the "accident_year" feature has a correlation value of "NaN." This might be due to limited variation in the "accident_year" feature, making it uninformative for predicting accident severity. We may consider excluding it from our analysis if it does not provide meaningful information.

# Decesion Tree Algorithm

In [15]:
# Select features and target variable
X = merged_data[['casualty_severity']]  # Features
y = merged_data['accident_severity']    # Target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a Decision Tree classifier
clf = DecisionTreeClassifier(random_state=42)

# Fit the model to the training data
clf.fit(X_train, y_train)

# Make predictions on the test data
y_pred = clf.predict(X_test) 

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

# Print accuracy
print(f"Accuracy: {accuracy:.2f}")

# Temporarily suppress the UndefinedMetricWarning
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=UndefinedMetricWarning)
    
    # Generate a classification report with zero_division parameter
    class_report = classification_report(y_test, y_pred, zero_division=0)
    print("Classification Report:\n", class_report)

# Restore warning settings to their original state
warnings.resetwarnings()

Accuracy: 0.94
Classification Report:
               precision    recall  f1-score   support

           1       1.00      0.63      0.77       947
           2       0.98      0.77      0.87     10585
           3       0.94      1.00      0.97     37723

    accuracy                           0.94     49255
   macro avg       0.97      0.80      0.87     49255
weighted avg       0.95      0.94      0.94     49255



# Logistic Regression 

After obtaining the relevant feature by using the corelation between the target variable and feature implemented the logistic regression algorithm on it. 

In [16]:

# Split the data into features (X) and the target variable (y)
X = merged_data[['casualty_severity']]
y = merged_data['accident_severity']

# Split the data into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train a logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test)

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:\n", classification_rep)


Accuracy: 0.9441884072682977
Classification Report:
               precision    recall  f1-score   support

           1       1.00      0.63      0.77       947
           2       0.98      0.77      0.87     10585
           3       0.94      1.00      0.97     37723

    accuracy                           0.94     49255
   macro avg       0.97      0.80      0.87     49255
weighted avg       0.95      0.94      0.94     49255



In [17]:

# Split the data into features (X) and the target variable (y)
X = merged_data[['casualty_class']]
y = merged_data['accident_severity']

# Split the data into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train a Support Vector Machine (SVM) model
model = SVC(kernel='linear') 
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test)

# Suppress the UndefinedMetricWarning
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    
    # Evaluate the model's performance
    accuracy = accuracy_score(y_test, y_pred)
    classification_rep = classification_report(y_test, y_pred)
    
print("Accuracy:", accuracy)
print("Classification Report:\n", classification_rep)


Accuracy: 0.7658714851284134
Classification Report:
               precision    recall  f1-score   support

           1       0.00      0.00      0.00       947
           2       0.00      0.00      0.00     10585
           3       0.77      1.00      0.87     37723

    accuracy                           0.77     49255
   macro avg       0.26      0.33      0.29     49255
weighted avg       0.59      0.77      0.66     49255



In [18]:
# Split data into train and test sets
X = merged_data[['weather_conditions']]  # Feature
y = merged_data['accident_severity']  # Target variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)  
rf_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf_classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(classification_rep)

Accuracy: 0.7658714851284134
              precision    recall  f1-score   support

           1       0.00      0.00      0.00       947
           2       0.00      0.00      0.00     10585
           3       0.77      1.00      0.87     37723

    accuracy                           0.77     49255
   macro avg       0.26      0.33      0.29     49255
weighted avg       0.59      0.77      0.66     49255



C:\Users\sayali\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sayali\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sayali\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
